<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2020</h1>
<h1 style="font-size: 250%;">Problem Set #2 </h1>
<h1>Issued Thursday, 2/11/2021; Due Thursday, 2/25/2021</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for up to two class meetings after the due date. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [7]:
# Replace the following string values with the requested information
class Student:
    first = "Katelyn"
    last = "Cline"
    onyen = "katelyn9"
    pid = "730152239"

You will need a the following sequence collection of gene promoter regions in which you will search for <a href="http://csbio.unc.edu/mcmillan/Comp555S21/motifs.fa" download="motifs.fa">transcription binding factor motifs</a>.

The cell below provides all functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In [8]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

---
**Problem #1:** In the cell below use the given *MedianStringMotifSearch()*, and *ScanAndScoreMotif()* functions to find an optimal 10-base motif pattern and its location in each of the 7 given promoter regions in <a href="http://csbio.unc.edu/mcmillan/Comp555S21/motifs.fa" download="motifs.fa">motifs.fa</a>. Based on this result, make a prediction of how long it would have taken to find an optimal 12-base motif (Hint: This will take some time).

In [6]:
# Read in the motifs.fa FASTA file and use its sequence 
# list as "dna" and run the given code fragment.
hdr, seq = loadFasta("data/motifs.fa")
dna = [s[1:] for s in seq]
#print(dna)

%time MedianStringMotifSearch(dna,10) 

Wall time: 3h 32min 39s


([874, 73, 113, 370, 5, 71, 189], 9, 'AGCCTGGGCA')

---
**Problem #2:** Given the optimal 10-base motif from **Problem #1** find an upper bound on the minHammingDist for the optimal 12-base motif. Explain how you arrived at your answer.

---
**Problem #3:** A simple optimization can be applied to *MedianStringMotifSearch()* as follows: If ever during the *ScanAndScoreMotif()* function the Hamming distance (i.e. *totalDist* in *ScanAndScoreMotif()*) exceeds the smallest Hamming distance seen thus far (*minHammingDist* in *MedianStringMotifSearch()*), scanning through subsequent sequences can be terminated early. Implement this strategy and use it to search again for the best 10-base motif. (Note: this requires that you add an additional *maxTotal* argument to the *ScanAndScoreMotif()*. You will need to modify both *MedianStringMotifSearch()* and *ScanAndScoreMotif()*). Run your new optimized version and report its run time. Revise your estimate of how how long it might take to find an optimal 12-base motif.

In [3]:
# Modify MedianStringMotifSearch() and call it BetterMedianStringMotifSearch().
# You will also need to modify ScanAndScoreMotif().
# Write these functions in this cell and run them using the following code fragment.

def ScanAndScoreMotif(DNA, motif, maxTotal):
    # Your code here
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            #this is the new bit
            # if the new hamming dist you just found is greater than the previously found min hamming dist
                #then you can return because the rest is irrelevant
            if (HammingDist > maxTotal):
                return bestAlignment, HammingDist
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def BetterMedianStringMotifSearch(DNA,k):
    # Your code here
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

#%time BetterMedianStringMotifSearch(dna,10)

The following cell below is provided for testing your code. You should test it on <a href="http://csbio.unc.edu/mcmillan/Comp555S21/motifs.fa" download="motifs.txt">motifs.fa</a>. This cell will not be considered during grading.

In [6]:
hdr, seq = loadFasta("data/motifs.fa")
dna = [s[1:] for s in seq]

%time BetterMedianStringMotifSearch(dna,10)

Wall time: 27min


([874, 73, 113, 370, 5, 71, 189], 9, 'AGCCTGGGCA')

**Problem #4:** Your upper-bound minHammingDist estimate from **Problem #2** can be used to further accelerate *MedianStringMotifSearch()* as follows. It can be used as the intitial setting of the minHammingDist rather than setting it to <code>k*len(DNA)</code> in addition to the changes made in **Problem #3**.  

In the cell provided below write *EvenBetterMedianMotifSearch(DNA,k,minHammingDist)*. Where the extra argument is used to set your upper-bound prediction.

In [4]:
# THIS CELL WILL BE GRADED
# This cell should include *ALL* functions called by EvenBetterMedianMotifSearch(),
# such as the modified ScanAndScoreMotif() from an earlier question.
# Any test or validation code should be placed in another cell 

def ScanAndScoreMotif(DNA, motif, maxTotal):
    # Your code here
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist > maxTotal):
                return bestAlignment, HammingDist
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def EvenBetterMedianMotifSearch(DNA,k,minHammingDist):
    bestAlignment = []
    kmer = ''
    # Add your code here
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

In [5]:
# This cell is provided for testing your answer to problem 4. It will not be graded.
hdr, seq = loadFasta("data/motifs.fa")
dna = [s[1:] for s in seq]
%time EvenBetterMedianMotifSearch(dna, 10, 19)

Wall time: 23min 38s


([874, 73, 113, 370, 5, 71, 189], 9, 'AGCCTGGGCA')

**Problem #5:** The cost of generating the upper-bound estimate in **Problem 2** was considerable. An alternative upper-bound can be found by running *ContainedMotifSearch()*, which considers only k-mers that appear in the given sequences as possible motifs. Use *ContainedMotifSearch()* with *k = 12* to establish an alternative upper bound. Which of these, (the one estimated in **Problem #2:**, or the one implied by *ContainedMotifSearch()* is tighter?

In [10]:
hdr, seq = loadFasta("data/motifs.fa")
dna = [s[1:] for s in seq]
%time ContainedMotifSearch(dna, 12)

3507 Motifs in our set
Wall time: 35.3 s


([331, 71, 159, 368, 121, 69, 163], 15, 'TGAGCCTGGGCA')

---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.